<a href="https://colab.research.google.com/github/kadefue/MoEST/blob/main/MoEST_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import os
import re
import warnings

# ==========================================
# 0. Setup: Block Warnings
# ==========================================
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

# ==========================================
# 1. Cleaning & Helper Functions
# ==========================================

def is_valid_text(value):
    """Checks if the value is Alphabetic or Alphanumeric."""
    s_val = str(value).strip()
    if not s_val:
        return False
    clean_val = s_val.replace(" ", "")
    if clean_val.isalnum():
        return True
    if re.search(r'[a-zA-Z]', s_val):
        return True
    return False

def normalize_merged_cells(df, header_rows=15):
    """
    Handles merged columns/rows in the header/label area.
    Duplicates text horizontally and vertically for merged cells.
    """
    if df.empty:
        return df

    limit = min(header_rows, len(df))
    subset = df.iloc[:limit].copy()

    # Forward fill horizontally and vertically
    subset = subset.ffill(axis=1)
    subset = subset.ffill(axis=0)

    df.iloc[:limit] = subset
    return df

def process_council_sheet(df):
    """
    Applies specific cleaning steps:
    1. Populate structural columns downwards.
    2. Remove 'Grand' and 'Total' rows.
    (Sparsity and Row checks are now moved to the final stage)
    """
    if df.empty:
        return df

    # Identify Region Column
    region_col = None
    for i, row in df.head(5).iterrows():
        for col in df.columns:
            val = str(row[col]).lower()
            if "region" in val or "mkoa" in val:
                region_col = col
                break
        if region_col is not None:
            break

    if region_col is None and not df.empty:
        region_col = df.columns[0]

    # Populate Columns Downwards (Unmerge Vertical for structural cols)
    cols_to_fill = list(df.columns[:3])
    if region_col is not None and region_col not in cols_to_fill:
        cols_to_fill.append(region_col)

    for col in cols_to_fill:
        if col in df.columns:
            df[col] = df[col].replace({0: None, '0': None})
            df[col] = df[col].ffill()

    # "Grand" Logic
    if region_col is not None and region_col in df.columns:
        grand_mask = df[region_col].astype(str).str.contains("Grand", case=False, na=False)
        if grand_mask.any():
            cutoff_idx = grand_mask.idxmax()
            df = df.loc[:cutoff_idx-1]

    # "Total" Logic (Region)
    if region_col is not None and region_col in df.columns:
        total_mask = df[region_col].astype(str).str.contains("Total", case=False, na=False)
        df = df[~total_mask]

    # "Total" Logic (Council)
    council_col = None
    council_keywords = ['council', 'halmashauri', 'district', 'lga', 'wilaya', 'municipal', 'town council']

    for i, row in df.head(5).iterrows():
        for col in df.columns:
            val = str(row[col]).lower()
            if any(kw in val for kw in council_keywords):
                council_col = col
                break
        if council_col is not None:
            break

    if council_col is not None and council_col in df.columns:
        pat = "Total|Sub-Total|Sub Total"
        council_total_mask = df[council_col].astype(str).str.contains(pat, case=False, na=False)
        df = df[~council_total_mask]

    # Final Cleanup: Remove rows with "Total" in first few columns
    target_indices = [0, 1, 2]
    for idx in target_indices:
        if idx < len(df.columns):
            col_name = df.columns[idx]
            mask = df[col_name].astype(str).str.contains("Total", case=False, na=False)
            df = df[~mask]

    # Duplicate Region Column Check
    if region_col is not None and region_col in df.columns:
        cols_to_drop = []
        for col in df.columns:
            if col == region_col: continue
            if df[col].equals(df[region_col]):
                cols_to_drop.append(col)
        if cols_to_drop:
            df = df.drop(columns=cols_to_drop)

    return df

def row_has_numeric(series):
    """Returns True if the row contains any numeric value."""
    for val in series:
        if isinstance(val, (int, float)) and not isinstance(val, bool):
            return True
        if isinstance(val, str):
            s = val.strip().replace(',', '')
            if s.replace('.', '', 1).isdigit():
                return True
    return False

def perform_final_cleanup(df):
    """
    Executes the final requested operations:
    1. Delete rows with only 1 cell of value (excluding the Source_Year column).
    2. Delete columns with >15% empty cells.
    """
    if df.empty:
        return df

    # --- 1. ROW CLEANUP ---
    # We check non-null count on all columns EXCEPT 'Source_Year'
    # If a row has <= 1 valid data cell, we drop it.
    cols_to_check = [c for c in df.columns if c != 'Source_Year']

    # Calculate non-nulls row-wise for data columns
    # We treat empty strings '' as Null here just in case
    temp_df = df[cols_to_check].replace('', None)
    row_counts = temp_df.notna().sum(axis=1)

    # Keep rows where we have MORE than 1 data value
    initial_rows = len(df)
    df = df[row_counts > 1]
    dropped_rows = initial_rows - len(df)
    if dropped_rows > 0:
        print(f"    (Cleaned {dropped_rows} rows having <= 1 data value)")

    # --- 2. COLUMN SPARSITY (15% Threshold) ---
    # Remove columns which have MORE than 15% empty cells
    threshold = 0.90
    initial_cols = len(df.columns)

    # Calculate null percentage
    # We treat 0 and '0' as valid values here? Usually yes, 0 is data.
    # But often empty strings are loaded as objects. Let's stick to standard NaNs/None.
    # If you consider '0' as empty, uncomment the replacement line below.
    # df_check = df.replace({0: None, '0': None, '': None})

    missing_pct = df.isna().mean()
    cols_to_keep = missing_pct[missing_pct <= threshold].index

    df = df[cols_to_keep]
    dropped_cols = initial_cols - len(df.columns)
    if dropped_cols > 0:
        print(f"    (Dropped {dropped_cols} columns with >15% empty cells)")

    return df

# ==========================================
# 2. Main Extraction and Combination Logic
# ==========================================

def extract_and_combine_all():
    base_dir = "/content/drive/MyDrive/BEST"

    # --- DEFINITION: Data Categories and Mappings ---

    # 1. Laboratories
    lab_mapping = {
        2016: ["3.26Lab", "3.27LabGov"],
        2017: ["3.36LabRegCoun", "3.37LabGovtRegCoun"],
        2018: ["3.37Lab", "3.38LabGov"],
        2019: ["3.37Lab", "3.38LabGov"],
        2020: ["3.37Lab", "3.38LabGov"],
        2021: ["T3.37Lab", "T3.38LabGov"],
        2022: ["T3.38Lab", "T3.39LabGov"],
        2023: ["T3.39LabG&NG", "T3.40LabG"],
        2024: ["T3.39LabG&NG", "T3.40LabG"],
        2025: ["T3.40LabG&NG", "T3.41LabG"]
    }

    # 2. ICT Equipment
    ict_mapping = {
        2017: ["T3.42ICTAllRegCoun", "T3.43ICTGovRegCoun"],
        2018: ["T2.42_ICT_G&N", "T2.43_ICT_G"],
        2019: ["Table170", "Table169"],
        2020: ["Table147", "Table148"],
        2021: ["Table155", "Table156"],
        2022: ["T2.43_ICT_G&N", "T2.44_ICT_G"],
        2023: ["T2.44_ICT_G&N", "T2.45_ICT_G"],
        2024: ["T2.44_ICT_G&N", "T2.45_ICT_G"],
        2025: ["T3.46ICT", "T3.47ICT_Gov"]
    }

    # 3. Electricity
    elec_mapping = {
        2017: ["T3.40SchElecAllRegCoun", "T3.41SchElecGovRegCoun"],
        2018: ["T2.44_Elect_G&N", "T2.45_Elect_G"],
        2019: ["Table165", "Table167"],
        2020: ["Table145", "Table146"],
        2021: ["Table152", "Table153"],
        2022: ["T2.41_Elect_G&N", "T2.42_Elect_G"],
        2023: ["T2.42_Elect_G&N", "T2.43_Elect_G"],
        2024: ["T2.42_Elect_G&N", "T2.43_Elect_G"],
        2025: ["T2.42_Elect_G&N", "T2.43_Elect_G"]
    }

    tasks = [
        ("Laboratories", lab_mapping, "Combined_Laboratories_All_G_NG.csv", "Combined_Laboratories_Govt.csv"),
        ("ICT_Equipment", ict_mapping, "Combined_ICT_All_G_NG.csv", "Combined_ICT_Govt.csv"),
        ("Electricity", elec_mapping, "Combined_Electricity_All_G_NG.csv", "Combined_Electricity_Govt.csv")
    ]

    for category_name, mapping, left_out, right_out in tasks:
        print(f"\n=======================================================")
        print(f" PROCESSING CATEGORY: {category_name}")
        print(f"=======================================================")

        left_dfs = []
        right_dfs = []

        processed_first_file = False

        for year, sheets in sorted(mapping.items()):
            filename = f"BEST {year}.xlsx"
            file_path = os.path.join(base_dir, filename)

            if not os.path.exists(file_path):
                print(f"Year {year}: File not found ({filename})")
                continue

            try:
                xls = pd.ExcelFile(file_path)
                available_sheets = xls.sheet_names

                # Normalize sheet names for robust lookup (ignore spaces)
                normalized_lookup = {s.replace(" ", ""): s for s in available_sheets}

                # Helper to process a single sheet
                def process_sheet_data(target_name_clean, is_first_time):
                    if target_name_clean in normalized_lookup:
                        real_sheet_name = normalized_lookup[target_name_clean]

                        # Load & Clean
                        df = pd.read_excel(xls, sheet_name=real_sheet_name, header=None)
                        df = normalize_merged_cells(df)
                        df = process_council_sheet(df)

                        # --- SMART ROW DELETION (Before adding Year) ---
                        status_suffix = ""
                        if not is_first_time:
                            check_limit = min(4, len(df))
                            top_slice = df.iloc[:check_limit]
                            rest_slice = df.iloc[check_limit:]

                            rows_to_keep = []
                            for idx in range(len(top_slice)):
                                row_data = top_slice.iloc[idx]
                                if row_has_numeric(row_data):
                                    rows_to_keep.append(top_slice.iloc[[idx]])

                            if rows_to_keep:
                                df = pd.concat(rows_to_keep + [rest_slice])
                                dropped_count = check_limit - len(rows_to_keep)
                                status_suffix = f"(Dropped {dropped_count} header rows, kept {len(rows_to_keep)} numeric rows)"
                            else:
                                df = rest_slice
                                status_suffix = f"(Dropped top {check_limit} header rows)"
                        else:
                            status_suffix = "(First file: All rows kept)"

                        # --- ADD YEAR COLUMN ---
                        df.insert(0, 'Source_Year', year)

                        print(f"  {year}: [FOUND] '{real_sheet_name}' {status_suffix}")
                        return df
                    else:
                        print(f"  {year}: [MISSING] '{target_name_clean}'")
                        return None

                # --- EXTRACT LEFT ---
                df_left = process_sheet_data(sheets[0], not processed_first_file)
                if df_left is not None:
                    left_dfs.append(df_left)

                # --- EXTRACT RIGHT ---
                if len(sheets) > 1:
                    df_right = process_sheet_data(sheets[1], not processed_first_file)
                    if df_right is not None:
                        right_dfs.append(df_right)

                # Mark success
                if df_left is not None or (len(sheets) > 1 and df_right is not None):
                    processed_first_file = True

            except Exception as e:
                print(f"  {year}: [ERROR] Processing file: {e}")

        # --- SAVE FILES FOR THIS CATEGORY ---
        print(f"\n--- Finalizing & Saving {category_name} ---")

        if left_dfs:
            combined_left = pd.concat(left_dfs, ignore_index=True)
            # PERFORM FINAL CLEANUP
            combined_left = perform_final_cleanup(combined_left)
            combined_left.to_csv(left_out, index=False, header=False)
            print(f"  -> Saved '{left_out}' ({len(combined_left)} rows)")
        else:
            print(f"  -> No data for '{left_out}'")

        if right_dfs:
            combined_right = pd.concat(right_dfs, ignore_index=True)
            # PERFORM FINAL CLEANUP
            combined_right = perform_final_cleanup(combined_right)
            combined_right.to_csv(right_out, index=False, header=False)
            print(f"  -> Saved '{right_out}' ({len(combined_right)} rows)")

    print("\n=== All Tasks Complete ===")

if __name__ == "__main__":
    extract_and_combine_all()


 PROCESSING CATEGORY: Laboratories
  2016: [FOUND] '3.26Lab' (First file: All rows kept)
  2016: [FOUND] '3.27LabGov' (First file: All rows kept)
  2017: [FOUND] '3.36LabRegCoun' (Dropped top 4 header rows)
  2017: [FOUND] '3.37LabGovtRegCoun' (Dropped top 4 header rows)
  2018: [FOUND] '3.37Lab' (Dropped top 4 header rows)
  2018: [FOUND] '3.38LabGov' (Dropped top 4 header rows)
  2019: [FOUND] '3.37Lab' (Dropped 2 header rows, kept 2 numeric rows)
  2019: [FOUND] '3.38LabGov' (Dropped 2 header rows, kept 2 numeric rows)
  2020: [FOUND] '3.37Lab' (Dropped 2 header rows, kept 2 numeric rows)
  2020: [FOUND] '3.38LabGov' (Dropped 2 header rows, kept 2 numeric rows)
  2021: [FOUND] 'T3.37Lab' (Dropped 2 header rows, kept 2 numeric rows)
  2021: [FOUND] 'T3.38LabGov' (Dropped 2 header rows, kept 2 numeric rows)
  2022: [FOUND] 'T3.38Lab' (Dropped top 4 header rows)
  2022: [FOUND] 'T3.39LabGov' (Dropped top 4 header rows)
  2023: [FOUND] 'T3.39LabG&NG' (Dropped top 4 header rows)
  2023: